In [1]:
import zipfile
import os

zip_path = "ssp4_3.4.zip"   # replace with your filename
extract_path = "cmip6_data"  # folder to extract into

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Files extracted:")
print(os.listdir(extract_path))

Files extracted:
['provenance.json', 'provenance.png', 'tos_Omon_GISS-E2-1-G_ssp434_r1i1p1f2_gn_20150116-21001216.nc']


In [3]:
which python
python --version
which pip

SyntaxError: invalid syntax (652831564.py, line 1)

In [2]:
!pip install xarray netCDF4 h5netcdf


  Using cached xarray-0.16.2-py3-none-any.whl (736 kB)
  Using cached netCDF4-1.6.2.tar.gz (777 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  ERROR: Command errored out with exit status 1:
   command: /Users/SophiaRahnke/opt/miniconda3/envs/py36test/bin/python /Users/SophiaRahnke/opt/miniconda3/envs/py36test/lib/python3.6/site-packages/pip/_vendor/pep517/_in_process.py get_requires_for_build_wheel /var/folders/cw/pns4_sf92n3gs1svfd1pwbhh0000gn/T/tmphxdtygz2
       cwd: /private/var/folders/cw/pns4_sf92n3gs1svfd1pwbhh0000gn/T/pip-install-ya0kd525/netCDF4
  Complete output (33 lines):
  reading from setup.cfg...
  
      HDF5_DIR environment variable not set, checking some standard locations ..
  checking /Users/SophiaRahnke/opt/miniconda3/envs/py36test/include ...
  HDF5 library version: 1.10.4 headers found in /Users/SophiaRahnke/opt/miniconda3/envs/py36test/include
  HDF5 library version: 1.10.4 found in /Users/SophiaRahnke/opt/minicond

In [3]:
!pip install dask cartopy


     |████████████████████████████████| 12.1 MB 779 kB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  ERROR: Command errored out with exit status 1:
   command: /Users/SophiaRahnke/opt/miniconda3/envs/py36test/bin/python /Users/SophiaRahnke/opt/miniconda3/envs/py36test/lib/python3.6/site-packages/pip/_vendor/pep517/_in_process.py get_requires_for_build_wheel /var/folders/cw/pns4_sf92n3gs1svfd1pwbhh0000gn/T/tmpo1z8ffi_
       cwd: /private/var/folders/cw/pns4_sf92n3gs1svfd1pwbhh0000gn/T/pip-install-oo_fcreo/cartopy
  Complete output (3 lines):
  setup.py:120: UserWarning: Unable to determine GEOS version. Ensure you have 3.3.3 or later installed, or installation may fail.
    '.'.join(str(v) for v in GEOS_MIN_VERSION), ))
  Proj 4.9.0 must be installed.
  ----------------------------------------
ERROR: Command errored out with exit status 1: /Users/SophiaRahnke/opt/miniconda3/envs/py36test/bin/python /Users/SophiaRahnke/opt/minic